# [Day 3: Spiral Memory](https://adventofcode.com/2017/day/3)

## Part A

Today's challenge was a much more interesting and difficult one than the previous two, with lots of possible different implementations.

Essentially we're dealing with a number spiral here:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23  24  ...
```

and for any number in the spiral, we'd like to know its [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) from 1. For instance, the distance for 12 is 3 (2 horizontally, 1 vertically).

Let's first take a look at a  "brute force" method, i.e we'll actually go and calculate out the spiral. Doing so seems somewhat complicated at first, but we can find a pretty simple pattern among the way the numbers are arranged: `1R, 1U, 2L, 2D, 3R, 3U, 4L, 4D, 5R, 5U, ...` where `3L` means you move left three times (and have a number in each spot). So the pattern for travel seems to generally be `move X, turn, move X, turn, move X+1, turn, move X+1, turn, ...`.

Let's make a Python [generator](https://wiki.python.org/moin/Generators) (think of it as a sequence that you call .next() on to get the next in the sequence) for the spiral indices. What we'll do is maintain our current spot in space `(x,y)`, a direction to travel `(dx,dy)` (e.g up would be `(0,1)`), and a distance to travel before turning, `distance_to_travel`. Whenever we turn to the left or to the right (i.e a turn to face a horizontal direction) we increase `distance_to_travel` by one.

In [1]:
def spiral_indices():
    # Helper for turning
    def turn_ccw(dx, dy):
        return (-dy, dx)
    
    (x, y) = (0, 0)   # Start at origin
    (dx, dy) = (1, 0) # Initial direction is rightwaesa
    distance_to_travel = 1
    distance_travelled = 0
    
    yield (x,y) # Return (0,0) the first time

    while(True):
        # Move
        (x, y) = (x + dx, y + dy)
        distance_travelled += 1
        
        # If we've reached the end of a segment turn, and if
        # we're now facing left/right, increment the distance
        # to travel
        if (distance_to_travel == distance_travelled):
            distance_travelled = 0
            (dx, dy) = turn_ccw(dx, dy)
            if (dy == 0):
                distance_to_travel += 1
        
        yield (x,y)

This code might seem really weird if you're unfamiliar with generators; after all isn't there an infinite loop? Basically, how a generator works is that when you call `next` on it, it executes the function until it first sees `yield X` and then returns X. The next time you call `next` it will continue "where it left off" (so the function execution doesn't start all over). It also works as you'd expect in things like for loops. Here's an example of our spiral generator in action:

In [2]:
gen = spiral_indices() # Initialize the generator
print(next(gen))
print(next(gen))
print(next(gen))

(0, 0)
(1, 0)
(1, 1)


In [3]:
for i in spiral_indices():
    print(i)
    if (i[0] > 1):
        break

(0, 0)
(1, 0)
(1, 1)
(0, 1)
(-1, 1)
(-1, 0)
(-1, -1)
(0, -1)
(1, -1)
(2, -1)


With this generator in place, finding the Manhattan distance of `n` is very simple.

In [4]:
def test_spirals(f):
    assert(f(1) == 0)
    assert(f(12) == 3)
    assert(f(23) == 2)
    assert(f(1024) == 31)

# Helper for finding a Manhattan distnace
def manhattan_dist(x, y):
    return abs(x) + abs(y)

def spirals(n):
    ct = 0
    for (x,y) in spiral_indices():
        ct += 1
        if ct == n:
            return manhattan_dist(x, y)

test_spirals(spirals)

## Part B

Now for Part B, we want to imagine the spiral as initially all 0's. Then we fill in the initial 1. The number in the next spiral location is then determined by summing up all its neighbors. So you can imagine the process as
```
000 000 002 042 542
010 011 011 011 011
000 000 000 000 000 ...
```
and the end result would be something that looks like
```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806  ...
```

We want the first value in the spiral that is larger than some number `n`.

To accomplish this we'll reuse the spiral_indices generator from Part A. We can keep track of an infinite grid initialized to 0 by using a dictionary mapping tuples `(i,j)` to the value at that location, and saying that the value is 0 if that tuple isn't in the dictionary yet. Then, as we iterate through the spiral we just need to properly update that dictionary. 

In [5]:
def test_spiral_sum(f):
    assert(f(1) == 2)
    assert(f(2) == 4)
    assert(f(5) == 10)
    assert(f(330) == 351)
    assert(f(500) == 747)

def spiral_sum(n):
    # Store the value at each location (i,j)
    locs = {}
    locs[(0,0)] = 1

    # Helper for accessing the location dictionary
    def get_loc(x,y):
        return locs[(x,y)] if (x,y) in locs else 0

    # Helper for calculating what the value of a location should be
    def record_loc(x,y):
        value = (get_loc(x-1,y+1) + get_loc(x,y+1) + get_loc(x+1,y+1) + 
                 get_loc(x-1,y)   +                  get_loc(x+1,y) + 
                 get_loc(x-1,y-1) + get_loc(x,y-1) + get_loc(x+1,y-1))
        locs[(x,y)] = value
        return value

    for (x, y) in spiral_indices():
        if x == 0 and y == 0: # Skip the initial index
            continue
        value = record_loc(x, y)
        if value > n:
            return value

test_spiral_sum(spiral_sum)

%timeit spiral_sum(1000)
%timeit spiral_sum(10000)
%timeit spiral_sum(100000)

63.5 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
90 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
132 µs ± 1.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Note that there are a few things that we could do to speed up the process for Part B. In particular, one simpleish optimization is to realize that in `record_loc`, we don't need to be checking every neighbor. For instance, if we know we're currently traveling up in the spiral we know everything to the right will be 0. If we're traveling left, than everything above us is 0. And so on. Thus we only actually need to look at 5 neighbors rather than 8.

Looking at the timing we can see that although it feels a bit slow, the time increase actually isn't that bad for larger `n` (due to the fact that the values being stored in the spiral get larger quite quickly).

## Alternatives for Part A

The spiral indices generator works out nicely for accomplishing both Part A and Part B, but there are plenty of tricks you can do in Part A to avoid such brute force generation of the entire spiral (and perhaps tricks for Part B as well, although I haven't figured out anything good yet).

For one, instead of looking at the spiral as a spiral, we can look at it as layers of squares. For example, the third layer would be

```
17  16  15  14  13
18              12
19              11
20              10
21  22  23  24  25

```

Note that the `ith` layer is a square with sides of length `2i-1` (since at each layer the side length increases by 2). This means that the total perimeter of each square is `4*(2i-3) + 4 = 8*(i-1)` (imagine the sides without the corners then adding the corners back in). We can also note that the bottom right corner of the `ith` layer will be `(2i-1)^2` (which makes perfect sense since the `ith` layer is a square with sides of length `2i-1` so the area, i.e the total number of numbers in the square, should be `(2i-1)^2`).

So... how does any of this actually help us? Well, note that if you're on the top or bottom side, then the layer square you're on minus 1 is exactly the y-distance (think if it as the number of layers you need to traverse) to the center. Similarly, if you're on the left or right side, the x-distance to the center is just your layer number minus 1. The layer that some number `n` is on is just `i` satisfying `(2(i-1)-1)^2 < n <= (2i-1)^2`. Think of this as being between two consecutive odd squares (since each odd square signals the "end" of a spiral layer). So since 15 is between the second odd square (9) and the third odd square (25), it's on the third layer.

Great, so finding the layer tells us one component/axis of the Manhattan distance; how do we figure out the other? Well, let's consider the difference between `n` and the bottom right of the current layer, i.e smallest odd square larger than `n`. This should be enough to tell us where along a side we are, and by considering the distance from where on a side we are and the center of a side we can get the other axis for our Manhattan distance. In other words, we can use the idea that `Manhattan distance = (layer - 1) + (distance to the center of side you're on)`.

Let's see what we can do once we've calculated `d = (2i-1)^2 - n` (where as before `i` is `n`'s layer). Well for one, we know that every corner is `side_length - 1 = 2(i-1)` apart. So `d % 2(i-1)` gives us our position along any side since every `2(i-1)` moves us to a corner, and then anything left over moves us along a side. As an example, consider 15; it's 10 away from 25, so the first 4 brings it to the bottom left corner, the next 4 to the top left corner, and then the remaining 2 moves it along the side. Now we just need to go from the position along the side to the position at the center of the side. In a side of length `2i-1`, if we count our positions as 0,1,...,2i-2 then clearly the center is just at `2(i-1)/2 = i-1`. So overall, if our position along a side is `p` then the distance to the center of the side we're on is just `abs(p - i + 1)`.

OK, so that was a lot of math, but the point is that we first want to calculate what layer we're on , `i`, by looking at what consecutive odd squares it's between. Then we can. `i-1` gives us the distance to the center along one axis. To get the other one, we look at `d = (2i-1)^2 - n` (distance from the bottom right corner). `p = d % 2(i-1)` tells us the position along a side and then `abs(p - i + 1)` tells us our distance from the center of the side, which is the distance to the center along the other axis. So without doing any spiral generation nonsense, we can solve Part A purely mathematically using the following algorithm: 

In [6]:
def find_layer(n):
    i = 2 # Start at layer 2 since we skipped 1
    while n > ((2*i - 1)*(2*i - 1)):
        i += 1
    return i

def distance_from_center_of_side(n, i):
    bottom_right = (2*i - 1)*(2*i - 1)
    dist_from_bottom_right = bottom_right - n
    side_length = 2*i - 1
    position_along_side = dist_from_bottom_right % (side_length - 1)
    center_position_along_side = i - 1
    return abs(position_along_side - center_position_along_side)

def spiral_math(n):
    # Special case for 1 since things are weird for a 1x1 square
    if n == 1:
        return 0
    
    layer = find_layer(n)
    distance_from_center_of_side(n, layer)
    
    return (layer - 1) + distance_from_center_of_side(n, layer)

test_spirals(spiral_math)

or if you just want a shorter chunk of math to speed things up (also note that `((2i - 1)*(2i - 1) - n) % 2(i-1)` simplifies to
```
(4i^2 - 4i + 1 - n) % 2(i-1) =
2i*2(i-1) + 1 - n % 2(i-1) = 
(1-n) % 2(i-1)
```

In [7]:
def spiral_math_short(n):
    if n == 1:
        return 0
    i = 2
    while n > (2*i - 1)*(2*i - 1):
        i += 1
    return (i - 1) + abs((1 - n) % (2*i - 2) - i + 1)

test_spirals(spiral_math_short)

This should be way faster than what we were doing before with manually generating the spiral. Some comparisons can be found below

In [8]:
%timeit spirals(100)
%timeit spiral_math_short(100)

%timeit spirals(1000)
%timeit spiral_math_short(1000)

%timeit spirals(10000)
%timeit spiral_math_short(10000)

23.6 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
956 ns ± 25.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
214 µs ± 8.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.47 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.03 ms ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.37 µs ± 97.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
